<a href="https://colab.research.google.com/github/pollinations/hive/blob/main/interesting_notebooks/Video2X.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<!-- ![Video2X Logo](https://user-images.githubusercontent.com/21986859/102733190-872a7880-4334-11eb-8e9e-0ca747f130b1.png) -->

# [Video2X](https://github.com/k4yt3x/video2x) Colab Notebook


| Algorithm | Implementation Used                                                 | Best-Suited Purpose | Remarks                                             |
| --------- | ------------------------------------------------------------------- | ------------------- | --------------------------------------------------- |
| waifu2x   | [waifu2x-ncnn-vulkan](https://github.com/nihui/waifu2x-ncnn-vulkan) | anime-styled arts   |                                                     |
| Anime4K   | N/A                                                                 | anime-styled arts   | much faster than waifu2x, but with lower quality    |
| SRMD      | [srmd-ncnn-vulkan](https://github.com/nihui/srmd-ncnn-vulkan)       | real-life arts      | faster than SRMD, but with lower quality            |
| RealSR    | [realsr-ncnn-vulkan](https://github.com/nihui/realsr-ncnn-vulkan)   | real-life arts      | very slow, but very impressive for real-life videos |


If you have any questions, suggestions, etc., post an issue [here](https://github.com/k4yt3x/video2x/issues/new).

Licensed under the ISC License: https://www.isc.org/licenses/. \
Copyright 2022 K4YT3X and contributors.

Last tested on February 9, 2022, commit [14f3cd1](https://github.com/k4yt3x/video2x/commit/14f3cd1e160ff5722bf0830137bdaff62c37f4b4).

## Step 0: Runtime Selection

This step is manual. You'll need to select a GPU runtime for Video2X to work. On the menu bar at the top, select:

> Runtime > Change runtime type > Hardware accelerator

Change "Hardware accelerator" to GPU and click "Save." After you're done, execute the following cell to see the detailed information about the GPU allocated to you (optional).

In [ ]:
#@title Inputs


video_file = ''  #@param {type: "string"}

# Set either width or height to a non-zero number. the other will be scaled accordingly
width = 0  #@param {type: "number"}

# Set either width or height to a non-zero number. the other will be scaled accordingly
height = 1080  #@param {type: "number"}

# The algorithm types are described in the model details below
algorithm = "srmd"  #@param ['waifu2x', 'srmd', 'realsr']

# Use video interpolation to double the video's framerate
double_framerate = False  #@param {type: "boolean"}


output_path = "/content"

In [ ]:
!nvidia-smi

## Step 1: Install Build Dependencies

This step installs the tools and libraries required to build and run Video2X.

In [ ]:
!apt-get update
!apt-get install -y --no-install-recommends \
    python3.8 python3-pip python3-opencv python3-pil python3-tqdm \
    python3-dev libvulkan-dev glslang-dev glslang-tools \
    build-essential swig git \
    mesa-vulkan-drivers ffmpeg ninja-build

## Step 2: Install Video2X and Python Dependencies

This step Video2X and its dependencies. Since some dependencies must be compiled from source code, it will take quite a while to complete. Sit back and have a coffee. This step might be drastically sped up in the future by including binary wheels for some of the dependencies in PyPI.

In [ ]:
# increase Git's cache size
# the default might be too small for rife's repo
!git config --global http.postBuffer 1048576000
!git config --global https.postBuffer 1048576000

# temporary: install rife-ncnn-vulkan-python directly from GitHub
#   since it is too large to upload to PyPI
# temporary: install video2x directly from GitHub
#   since version 5.0.0 doesn't have a release on PyPI yet
!python3.8 -m pip install -U pip setuptools setuptools-scm wheel
!python3.8 -m pip install \
    rife-ncnn-vulkan-python@git+https://github.com/media2x/rife-ncnn-vulkan-python.git \
    video2x@git+https://github.com/k4yt3x/video2x.git@master

# upgrade Colab'd default Pillow (too old)
!python3.8 -m pip install -U Pillow

## Step 3: Mount Google Drive

In order to read/write video data from/to your Google Drive, you will need to grant Google Colab access to your Google Drive. Upon running the following script, you'll be prompted to permit this script access to your Google Drive.

In [ ]:
# mount your Google Drive into the runtime
#from google.colab import drive
#drive.mount('/mnt/gdrive')

# change the working directory to the root directory of your Google Drive
#%cd /mnt/gdrive/MyDrive/

## Step 4: Launch Video2X

You can launch Video2X now. **Remember to use the free resource fairly and not to run a free session for over 12 hours.** You can get [Colab Pro/Pro+](https://colab.research.google.com/signup/pricing) for longer runtimes.

The example command below upscales the file `input.mp4` in your Google Drive to height 1440 and saves the output file to `output.mp4`. You can specify either the width or height. The other paramter will be calculated automatically based on the input file's apsect ratio.

In [ ]:
from pathlib import Path
filename = Path('/root/dir/sub/file.ext').stem
superres_file = f"{output_path}/{filename}_1upscaled.mp4"

width_param = ""
height_param = ""

if width > 0:
  width_param = f"-w {width}"

if height > 0:
  height_param = f"-w {height}"


!python3.8 -m video2x -i {video_file} -o {superres_file} -p5 upscale {width_param} {height_param} -a {algorithm} -n3

The following command performs frame interpolation and 2x the frame rate of the video `input.mp4` and saves the output video to `output.mp4`. Currently, interpolation only supports 2x frame rate.

In [ ]:
if double_framerate:
  interp_file = f"{output_path}/{filename}_2_upscaled_interpolated.mp4"
  !python3.8 -m video2x -i {superres_file} -o {interp_file} interpolate

In [ ]:
from google.colab import drive
drive.mount('/gdrive')